In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
dest = '/content/drive/MyDrive/HSE Final Project/to_train'
name = 'submit_5_abandoned.csv'

df = pd.read_csv(dest + '/' + name)
df.head()

,shop_id,item_id,year,date_block_num,item_cnt,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,city_enc,shop_type_enc,item_category_id,digital,item_area_enc,area_to_enc
0,0,19,2013,0,0.0,28.0,-1.0,-1.0,-1.0,-1.0,29,4,40,0,7,4
1,0,27,2013,0,0.0,2499.0,-1.0,-1.0,-1.0,-1.0,29,4,19,0,5,9
2,0,28,2013,0,0.0,549.0,-1.0,-1.0,-1.0,-1.0,29,4,30,0,5,49
3,0,29,2013,0,0.0,2499.0,-1.0,-1.0,-1.0,-1.0,29,4,23,0,5,18
4,0,32,2013,0,6.0,349.0,-1.0,-1.0,-1.0,-1.0,29,4,40,0,7,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10913804 entries, 0 to 10913803
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   shop_id           int64  
 1   item_id           int64  
 2   year              int64  
 3   date_block_num    int64  
 4   item_cnt          float64
 5   item_price        float64
 6   prev_value        float64
 7   prev_value_year   float64
 8   stock_prev        float64
 9   stock_prev_year   float64
 10  city_enc          int64  
 11  shop_type_enc     int64  
 12  item_category_id  int64  
 13  digital           int64  
 14  item_area_enc     int64  
 15  area_to_enc       int64  
dtypes: float64(6), int64(10)
memory usage: 1.3 GB


In [4]:
X = df.drop('item_cnt', axis=1)
y = df.loc[:, 'item_cnt']

Due to the fact that we predict future, it makes no sense to shuffle the data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=False,
                                                    random_state=7)

In [6]:
learning_rate = 0.1
max_depth = 7
random_state = 7
verbosity = 2

In [7]:
xgbr = xgb.XGBRegressor(verbosity=verbosity,
                        eta=learning_rate,
                        max_depth=max_depth,
                        random_state=random_state)

In [8]:
xgbr.fit(X_train, y_train)

[13:45:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:45:20] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[13:45:51] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=7
[13:46:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 0 pruned nodes, max_depth=7
[13:46:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[13:47:06] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=7
[13:47:32] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 230 extra nodes, 0 pruned nodes, max_depth=7
[13:47:58

XGBRegressor(eta=0.1, max_depth=7, random_state=7, verbosity=2)

In [9]:
from sklearn.metrics import mean_squared_error

y_pred = xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('MSE: %.2f' % mse)
print('RMSE: %.2f' % (mse ** (0.5)))

MSE: 10.88
RMSE: 3.30


In [10]:
# from sklearn.metrics import mean_squared_log_error
# msle = mean_squared_log_error(y_test, y_pred)
# print('RMSLE: %.2f' % (msle ** (0.5)))

In [11]:
dest = '/content/drive/MyDrive/HSE Final Project/to_test'
name = 'submit_5_test.csv'

test_df = pd.read_csv(dest + '/' + name)
test_df.head()

,shop_id,item_id,year,date_block_num,item_price,prev_value,prev_value_year,stock_prev,stock_prev_year,city_enc,shop_type_enc,item_category_id,digital,item_area_enc,area_to_enc
0,5,5037,2015,34,2599.0,0.0,1.0,0.0,2599.0,3,3,19,0,5,9
1,5,5320,2015,34,-1.0,-1.0,-1.0,1.0,1.0,3,3,55,0,9,2
2,5,5233,2015,34,1199.0,1.0,-1.0,1199.0,-1.0,3,3,19,0,5,9
3,5,5232,2015,34,1199.0,0.0,-1.0,0.0,-1.0,3,3,23,0,5,18
4,5,5268,2015,34,-1.0,-1.0,-1.0,1.0,1.0,3,3,20,0,5,10


In [12]:
test_pred = xgbr.predict(test_df)
print(test_pred)

[ 0.28853595 -0.01055622  0.6059671  ...  0.09267887  0.07860407
  0.0634844 ]


In [13]:
name_sample = 'sample_submission.csv'

sample = pd.read_csv(dest + '/' + name_sample)
sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [14]:
sample.item_cnt_month = np.round(test_pred)
sample.head()

,ID,item_cnt_month
0,0,0.0
1,1,-0.0
2,2,1.0
3,3,0.0
4,4,0.0


In [15]:
dest = '/content/drive/MyDrive/HSE Final Project/submissions/'
sample.to_csv(dest + 'submission #9 (data from categories).csv', index=False)